In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-prediction-dataset/data_descriptions.csv
/kaggle/input/churn-prediction-dataset/train.csv
/kaggle/input/churn-prediction-dataset/test.csv


Import libraries

In [7]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load dataset
df = pd.read_csv('/kaggle/input/churn-prediction-dataset/train.csv')

Confidence interval (95%)
-

In [8]:
# Select category variables
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNumeric variables ({len(numeric_cols)}):\n", numeric_cols)

# Function to calculate category variables
def confidence_interval(series, confidence=0.95):
    data = series.dropna()
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    se = std / np.sqrt(n)
    alpha = 1 - confidence
    t_crit = stats.t.ppf(1 - alpha/2, df=n-1)
    margin = t_crit * se
    lower = mean - margin
    upper = mean + margin

    return pd.Series({
        "Sample Size (n)": n,
        "Mean (x̄)": mean,
        "Std Dev (s)": std,
        "Std Error": se,
        "t-critical": t_crit,
        "Margin of Error": margin,
        "95% CI Lower": lower,
        "95% CI Upper": upper
    })

# Apply for all var
ci_results = df[numeric_cols].apply(confidence_interval).T

# Display the result
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")
display(ci_results)


Numeric variables (10):
 ['AccountAge', 'MonthlyCharges', 'TotalCharges', 'ViewingHoursPerWeek', 'AverageViewingDuration', 'ContentDownloadsPerMonth', 'UserRating', 'SupportTicketsPerMonth', 'WatchlistSize', 'Churn']


,Sample Size (n),Mean (x̄),Std Dev (s),Std Error,t-critical,Margin of Error,95% CI Lower,95% CI Upper
AccountAge,"243,787.000",60.084,34.285,0.069,1.960,0.136,59.948,60.220
MonthlyCharges,"243,787.000",12.491,4.328,0.009,1.960,0.017,12.474,12.508
TotalCharges,"243,787.000",750.741,523.073,1.059,1.960,2.076,748.665,752.817
ViewingHoursPerWeek,"243,787.000",20.502,11.244,0.023,1.960,0.045,20.458,20.547
AverageViewingDuration,"243,787.000",92.264,50.505,0.102,1.960,0.200,92.064,92.465
ContentDownloadsPerMonth,"243,787.000",24.504,14.421,0.029,1.960,0.057,24.446,24.561
UserRating,"243,787.000",3.003,1.155,0.002,1.960,0.005,2.998,3.007
SupportTicketsPerMonth,"243,787.000",4.504,2.873,0.006,1.960,0.011,4.493,4.516
WatchlistSize,"243,787.000",12.019,7.193,0.015,1.960,0.029,11.990,12.047
Churn,"243,787.000",0.181,0.385,0.001,1.960,0.002,0.180,0.183



 Independent two-sample t-test to find the relationship between 2 groups churn and non-churn.
 Levene's test for the relationship between category variables.
 -
 Compare and display statisticable results.

In [11]:
# Select numeric and category variables
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Churn' in numeric_cols:
    numeric_cols.remove('Churn')

categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"Numeric variables ({len(numeric_cols)}): {numeric_cols}")
print(f"Categorical variables ({len(categorical_cols)}): {categorical_cols}")

# Devide into 2 groups
churned = df[df['Churn'] == 1]
non_churned = df[df['Churn'] == 0]

Numeric variables (9): ['AccountAge', 'MonthlyCharges', 'TotalCharges', 'ViewingHoursPerWeek', 'AverageViewingDuration', 'ContentDownloadsPerMonth', 'UserRating', 'SupportTicketsPerMonth', 'WatchlistSize']
Categorical variables (11): ['SubscriptionType', 'PaymentMethod', 'PaperlessBilling', 'ContentType', 'MultiDeviceAccess', 'DeviceRegistered', 'GenrePreference', 'Gender', 'ParentalControl', 'SubtitlesEnabled', 'CustomerID']

===================== 🧪 T-TEST + LEVENE RESULTS =====================


,Mean_Churned,Mean_NonChurned,Difference,Levene_p,Equal Variance?,t-statistic,p-value,Significant? (α=0.05)
AccountAge,45.6741,63.2733,-17.5992,0.0000,No,-103.4273,0.0000,Yes
MonthlyCharges,13.4149,12.2861,1.1288,0.0000,No,50.3551,0.0000,Yes
TotalCharges,616.7378,780.4022,-163.6644,0.0000,No,-62.0708,0.0000,Yes
ViewingHoursPerWeek,17.4277,21.1827,-3.7550,0.0000,No,-65.0389,0.0000,Yes
AverageViewingDuration,76.4948,95.7545,-19.2597,0.0000,No,-75.0632,0.0000,Yes
ContentDownloadsPerMonth,20.5263,25.3839,-4.8575,0.0000,No,-65.5738,0.0000,Yes
SupportTicketsPerMonth,5.0175,4.3906,0.6269,0.0000,No,41.8525,0.0000,Yes
UserRating,3.0570,2.9907,0.0663,0.3655,Yes,10.9261,0.0000,Yes
WatchlistSize,12.3509,11.9449,0.4059,0.2280,Yes,10.7359,0.0000,Yes



===================== 🔢 CHI-SQUARE RESULTS =====================


,Chi2 Statistic,p-value,DoF,Significant? (α=0.05)
SubscriptionType,318.4494,0.0000,2,Yes
PaymentMethod,239.8638,0.0000,3,Yes
GenrePreference,169.2967,0.0000,4,Yes
ContentType,43.4540,0.0000,2,Yes
SubtitlesEnabled,37.2516,0.0000,1,Yes
Gender,13.5542,0.0002,1,Yes
ParentalControl,7.4292,0.0064,1,Yes
DeviceRegistered,2.8391,0.4171,3,No
CustomerID,"243,787.0000",0.4990,243786,No
MultiDeviceAccess,0.3962,0.5291,1,No


In [ ]:
# Apply T-test and Levene's test
def t_test_by_group(var):
    x1 = churned[var].dropna()
    x2 = non_churned[var].dropna()

    # Levene's Test for equal variances
    _, p_levene = stats.levene(x1, x2)
    equal_var = p_levene > 0.05  # True nếu phương sai bằng nhau

    # Independent samples t-test
    t_stat, p_value = stats.ttest_ind(x1, x2, equal_var=equal_var)

    return pd.Series({
        'Mean_Churned': x1.mean(),
        'Mean_NonChurned': x2.mean(),
        'Difference': x1.mean() - x2.mean(),
        'Levene_p': p_levene,
        'Equal Variance?': 'Yes' if equal_var else 'No',
        't-statistic': t_stat,
        'p-value': p_value,
        'Significant? (α=0.05)': 'Yes' if p_value < 0.05 else 'No'
    })

# T-test for numeric var
t_test_results = pd.DataFrame({var: t_test_by_group(var) for var in numeric_cols}).T

In [13]:
pd.set_option('display.float_format', lambda x: f"{x:,.4f}")

print("\n===================== T-TEST + LEVENE RESULTS =====================")
display(t_test_results.sort_values('p-value'))


===================== T-TEST + LEVENE RESULTS =====================


,Mean_Churned,Mean_NonChurned,Difference,Levene_p,Equal Variance?,t-statistic,p-value,Significant? (α=0.05)
AccountAge,45.6741,63.2733,-17.5992,0.0000,No,-103.4273,0.0000,Yes
MonthlyCharges,13.4149,12.2861,1.1288,0.0000,No,50.3551,0.0000,Yes
TotalCharges,616.7378,780.4022,-163.6644,0.0000,No,-62.0708,0.0000,Yes
ViewingHoursPerWeek,17.4277,21.1827,-3.7550,0.0000,No,-65.0389,0.0000,Yes
AverageViewingDuration,76.4948,95.7545,-19.2597,0.0000,No,-75.0632,0.0000,Yes
ContentDownloadsPerMonth,20.5263,25.3839,-4.8575,0.0000,No,-65.5738,0.0000,Yes
SupportTicketsPerMonth,5.0175,4.3906,0.6269,0.0000,No,41.8525,0.0000,Yes
UserRating,3.0570,2.9907,0.0663,0.3655,Yes,10.9261,0.0000,Yes
WatchlistSize,12.3509,11.9449,0.4059,0.2280,Yes,10.7359,0.0000,Yes


Chi-square Test: test the relationship between 'churn' and category variables
-

p-value < 0.05 → Bác bỏ H₀ → Biến có sự khác biệt có ý nghĩa thống kê giữa nhóm churn và non-churn.

p-value ≥ 0.05 → Không bác bỏ H₀ → Không có sự khác biệt rõ rệt.

In [12]:
chi2_results = {}

for col in categorical_cols:
    try:
        ct = pd.crosstab(df[col], df['Churn'])
        chi2, p, dof, ex = stats.chi2_contingency(ct)
        chi2_results[col] = {'Chi2 Statistic': chi2, 'p-value': p, 'DoF': dof, 
                             'Significant? (α=0.05)': 'Yes' if p < 0.05 else 'No'}
    except Exception as e:
        chi2_results[col] = {'Chi2 Statistic': np.nan, 'p-value': np.nan, 'DoF': np.nan, 'Significant? (α=0.05)': 'Error'}

chi2_df = pd.DataFrame(chi2_results).T

In [14]:

print("\n===================== CHI-SQUARE RESULTS =====================")
display(chi2_df.sort_values('p-value'))


===================== CHI-SQUARE RESULTS =====================


,Chi2 Statistic,p-value,DoF,Significant? (α=0.05)
SubscriptionType,318.4494,0.0000,2,Yes
PaymentMethod,239.8638,0.0000,3,Yes
GenrePreference,169.2967,0.0000,4,Yes
ContentType,43.4540,0.0000,2,Yes
SubtitlesEnabled,37.2516,0.0000,1,Yes
Gender,13.5542,0.0002,1,Yes
ParentalControl,7.4292,0.0064,1,Yes
DeviceRegistered,2.8391,0.4171,3,No
CustomerID,"243,787.0000",0.4990,243786,No
MultiDeviceAccess,0.3962,0.5291,1,No


** BONUS: ADVANCED ANOVA PIPELINE (ONE-WAY, TWO-WAY, POST-HOC)
-
display the results as tables for easy-compare.

In [ ]:
# ADVANCED ANOVA PIPELINE (ONE-WAY, TWO-WAY, POST-HOC)

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats

# PHÂN LOẠI BIẾN
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# Nếu biến mục tiêu 'Churn' là numeric, ta bỏ nó ra
if 'Churn' in numeric_cols:
    numeric_cols.remove('Churn')

# ONE-WAY ANOVA CHO TỪNG CẶP (num × cat)
anova_results = []

for num in numeric_cols:
    for cat in categorical_cols:
        try:
            groups = [df[df[cat] == c][num].dropna() for c in df[cat].dropna().unique()]
            if len(groups) >= 2:  # chỉ xét nếu có ít nhất 2 nhóm
                f_stat, p_val = stats.f_oneway(*groups)
                anova_results.append({
                    'Numeric_Var': num,
                    'Categorical_Var': cat,
                    'F-statistic': f_stat,
                    'p-value': p_val
                })
        except Exception as e:
            pass

anova_df = pd.DataFrame(anova_results)
anova_df['Significant?'] = np.where(anova_df['p-value'] < 0.05, 'Yes', 'No')

print("✅ ONE-WAY ANOVA Results:")
display(anova_df.sort_values('p-value').reset_index(drop=True))


# TWO-WAY ANOVA (kết hợp 2 biến phân loại ảnh hưởng đến 1 biến số)
two_way_results = []

for num in numeric_cols:
    for i in range(len(categorical_cols)):
        for j in range(i + 1, len(categorical_cols)):
            cat1, cat2 = categorical_cols[i], categorical_cols[j]
            try:
                model = ols(f'{num} ~ C({cat1}) * C({cat2})', data=df).fit()
                anova_table = sm.stats.anova_lm(model, typ=2)
                p_main1 = anova_table.loc[f'C({cat1})', 'PR(>F)']
                p_main2 = anova_table.loc[f'C({cat2})', 'PR(>F)']
                p_interaction = anova_table.loc[f'C({cat1}):C({cat2})', 'PR(>F)']
                two_way_results.append({
                    'Numeric_Var': num,
                    'Cat1': cat1,
                    'Cat2': cat2,
                    'p_Cat1': p_main1,
                    'p_Cat2': p_main2,
                    'p_Interaction': p_interaction
                })
            except Exception as e:
                pass

two_way_df = pd.DataFrame(two_way_results)
print("\n✅ TWO-WAY ANOVA Results:")
display(two_way_df.sort_values('p_Interaction').reset_index(drop=True))


# 4️⃣ POST-HOC TEST (Tukey HSD) cho các cặp biến có ý nghĩa
posthoc_results = []

for idx, row in anova_df.iterrows():
    if row['p-value'] < 0.05:  # chỉ chạy post-hoc nếu ANOVA có ý nghĩa
        num = row['Numeric_Var']
        cat = row['Categorical_Var']
        try:
            tukey = pairwise_tukeyhsd(endog=df[num], groups=df[cat], alpha=0.05)
            tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
            tukey_df['Numeric_Var'] = num
            tukey_df['Categorical_Var'] = cat
            posthoc_results.append(tukey_df)
        except Exception as e:
            pass

if posthoc_results:
    posthoc_df = pd.concat(posthoc_results, ignore_index=True)
    print("\n POST-HOC (Tukey HSD) Results for Significant ANOVAs:")
    display(posthoc_df)
else:
    print("\n No significant ANOVA results found for post-hoc testing.")
